In [3]:
!pip install -q -U kaggle_environments

In [2]:
from kaggle_environments import make, evaluate
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Бот «Бумага»

In [12]:
%%writefile bot_paper.py
import random
def your_agent(obs, config):
    return 0

Overwriting bot_paper.py


In [10]:
evaluate("rps", ["bot_paper.py",  "statistical"], configuration={"episodeSteps": 100})

[[-98.0, 98.0]]

# Бот «Камень»

In [14]:
%%writefile bot_rock.py
import random
def your_agent(obs, config):
    return 1

Overwriting bot_rock.py


In [15]:
evaluate("rps",["bot_rock.py",  "statistical"], configuration={"episodeSteps": 100})

[[-97.0, 97.0]]

# Бот «Ножницы»

In [16]:
%%writefile bot_noj.py
import random
def your_agent(observation, configuration):
    return 2

Writing bot_noj.py


In [17]:
evaluate("rps",["bot_noj.py",  "statistical"],configuration={"episodeSteps": 100})

[[-99.0, 99.0]]

# Кастомный бот

In [18]:
%%writefile bot_custom.py
import random
def your_agent(obs, config):
    return random.randrange(random.randrange(0,2), 3)

Writing bot_custom.py


In [19]:
evaluate("rps",["bot_custom.py",  "paper"], configuration={"episodeSteps": 100})

[[0, 0]]

# Бот оппонент

In [20]:
%%writefile bot_oppon.py
import random
def your_agent(obs, config):
    if obs.step > 0:
        return [0,1,2][obs.lastOpponentAction-2]
    else:
         return random.randrange(0, config.signs)

Writing bot_oppon.py


In [21]:
evaluate("rps",["bot_oppon.py",  "statistical"], configuration={"episodeSteps": 100})

[[57.0, -57.0]]

# Бот неслучайный

In [22]:
numbers = [0,1,2]
numbers.remove(random.choice(numbers))
random.choice(numbers)

2

In [23]:
%%writefile not_casion.py
import random
def your_agent(obs, config):
    if obs.step > 0:
        move = [0,1,2]
        move.remove(obs.lastOpponentAction)
        return random.choice(move)
    else:
         return random.randrange(0, config.signs)

Writing not_casion.py


In [24]:
evaluate("rps",["not_casion.py",  "statistical"],configuration={"episodeSteps": 100})

[[23.0, -23.0]]

# Рандомные боты (шаг != num%2==0)

In [33]:
%%writefile paper_2.py
import random
def your_agent(obs, config):
    if not obs.step%2: return 1
    else: 
        return random.choice([0, 2])

Overwriting paper_2.py


In [35]:
evaluate("rps",["paper_2.py",  "statistical"],configuration={"episodeSteps": 100})

[[-25.0, 25.0]]

In [27]:
%%writefile rock_2.py
import random
def your_agent(obs, config):
    if not obs.step%2: return 0
    else: 
        return random.choice([1, 2])

Writing rock_2.py


In [28]:
evaluate("rps",["rock_2.py",  "statistical"],configuration={"episodeSteps": 100})

[[-24.0, 24.0]]

In [29]:
%%writefile noj_2.py
import random
def your_agent(obs, config):
    if not obs.step%2: return 2
    else: 
        return random.choice([0, 1])

Writing noj_2.py


In [24]:
evaluate(
    "rps",["noj_2.py",  "statistical"],configuration={"episodeSteps": 100})

[[-29.0, 29.0]]

# Рандомные боты (шаг = num%2==0)

In [36]:
%%writefile random_1_1.py
import random
def your_agent(obs, config):
    if obs.step > 0:
        if not obs.step%2: 
            return obs.lastOpponentAction      
        else:
            return [0,1,2][obs.lastOpponentAction-2]
    else:
         return random.randrange(0, config.signs)


Writing random_1_1.py


In [37]:
evaluate("rps",["random_1_1.py",  "statistical"],configuration={"episodeSteps": 100})

[[0, 0]]

In [38]:
%%writefile random_1_2.py
import random
def your_agent(obs, config):
    if obs.step > 0:
        if not obs.step%2: 
            return obs.lastOpponentAction      
        else:
            return [0,1,2][obs.lastOpponentAction-2]
    else:
         return random.randrange(0, config.signs)

Writing random_1_2.py


In [39]:
evaluate("rps",["random_1_2.py",  "statistical"],configuration={"episodeSteps": 100})

[[96.0, -96.0]]

# Бот НЕ по четности шага

In [40]:
%%writefile not_random_1.py
import random
def your_agent(obs, config):
    if obs.step > 0:
        if not obs.step%2:
          for_action = [0,1,2]
          for_action.remove(obs.lastOpponentAction)
          return random.choice(for_action)  
        else:
            return [0,1,2][obs.lastOpponentAction-2]
    else:
         return random.randrange(0, config.signs)

Writing not_random_1.py


In [41]:
evaluate("rps",["not_random_1.py",  "statistical"],configuration={"episodeSteps": 100})

[[39.0, -39.0]]

# Бот, запоминающий выбор

In [42]:
%%writefile bot_mind.py
import random
mem_opponent = {}
def your_agent(obs, config):
    global mem_opponent
    if obs.step > 0:
        mem_opponent[obs.lastOpponentAction] = mem_opponent.get(obs.lastOpponentAction, 0)+1
        friq_step_opponent = sorted(mem_opponent, key=mem_opponent.get, reverse=True)[0]
        return [0,1,2][friq_step_opponent-2]
    else:
         return random.randrange(0, config.signs)

Writing bot_mind.py


In [43]:
evaluate("rps",["bot_mind.py",  "statistical"],configuration={"episodeSteps": 100})

[[0, 0]]

# Анти-бот, запоминающий выбор

In [44]:
%%writefile bot_mind_2.py
import random
mem_opponent = {}
def your_agent(obs, config):
    global mem_opponent
    if obs.step > 0:
        mem_opponent[obs.lastOpponentAction] = mem_opponent.get(obs.lastOpponentAction, 0)+1
        friq_step_opponent_sorted = sorted(mem_opponent, key=mem_opponent.get)
        prob_step_sorted = [mem_opponent[key]/obs.step for key in friq_step_opponent_sorted]
        bias = random.random()
        for i, prob in enumerate(prob_step_sorted):
          if prob > bias: break
        return [0,1,2][friq_step_opponent_sorted[i]-2]
    else:
         return random.randrange(0, config.signs)

Writing bot_mind_2.py


# Турнир

Организован вариативный турнир между участниками разбиение на случайные пары - если четное количество и четное к-во пар или груговой турнир определяется на кадом этапе

In [45]:
def get_pairs(lst, n): # разделяем список на n элементов
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def round_tournirs(voriers): # запускаем круговой тур
    qty_voriers = len(voriers)
    scores = [0 for _ in range(qty_voriers)]
    for x in range(qty_voriers):
      for y in range(x+1, qty_voriers):
        for z in range(PERIODS):
            res = evaluate("rps", [voriers[x], voriers[y]],configuration={'episodeSteps': 50,'tieRewardThreshold':1})[0]
            if not period: results.append(res)
            else:
              scores[voriers.index(voriers[x])]+= res[0]
              scores[voriers.index(voriers[y])]+= res[1]
    sort_scores = sorted(scores)          
    voriers = [voriers[scores.index(score)] for score in sort_scores]
    return voriers[qty_voriers//2:]

In [51]:
voriers = ['rock',
           'paper',
           'scissors',
           'copy_opponent',
           'reactionary',
           'counter_reactionary',
           'bot_paper.py', # 1
           'bot_rock.py', # 2
           'bot_noj.py', # 3
           'bot_custom.py', # 4
           'bot_oppon.py', # 5
           'not_casion.py', # 6
           'paper_2.py', # 7
           'rock_2.py', # 8
           'noj_2.py', # 9
           'random_1_1.py', # 10
           'random_1_2.py', # 11
           'not_random_1.py', # 12
           'bot_mind.py', # 13
           'bot_mind_2.py' # 14
]

PERIODS = 6 # задаем периоды парных игр
random.shuffle(voriers) # рандомозируем участников
qty_voriers = len(voriers)
PLAY = True
step = 1

while PLAY:
    if qty_voriers%2:
      print('Нечетное кол-во ботов, новый турнир:')
      print('WAIT... /n')
      voriers = round_tournirs(voriers) # выбираем победителя

    else:
        pairs = qty_voriers//2
        if not pairs%2:
            print('Выбираем пары рандомно')
            print('WAIT...')
            groups = list(get_pairs(voriers, 2))
            print(f'Кол-во пар {len(groups)}')
            results = []
            for period in range(PERIODS):
              for i, pair in enumerate(groups):
                res = evaluate("rps",pair,configuration={'episodeSteps': 100,'tieRewardThreshold':1})[0]
                if not period: results.append(res)
                else:
                  results[i][0]+=res[0]
                  results[i][1]+=res[1]
              voriers = []
              for result, pair in zip(results, groups):
                  if result[0]>result[1]: voriers.append(pair[0])
                  elif result[0]<result[1]: voriers.append(pair[1])
                  else: voriers.extend(pair)

        else:
          print('Нечетное кол-во ботов, новый турнир:')
          print('WAIT.../n')
          voriers = round_tournirs(voriers)

    print(f'Победители круга {step}')
    for vorier in voriers:
      print(vorier)
    print()
    random.shuffle(voriers)
    step+=1

    qty_voriers = len(voriers)
    if qty_voriers<2:
      print(f'Winner: {voriers[0]}')
      PLAY = False

Выбираем пары рандомно
WAIT...
Кол-во пар 10
Победители круга 1
bot_mind.py
copy_opponent
bot_mind_2.py
counter_reactionary
rock_2.py
reactionary
paper
not_random_1.py
noj_2.py
bot_custom.py

Нечетное кол-во ботов, новый турнир:
WAIT.../n/n
Победители круга 2
bot_mind.py
copy_opponent
noj_2.py
bot_mind_2.py
rock_2.py

Нечетное кол-во ботов, новый турнир:
WAIT... /n/n
Победители круга 3
rock_2.py
bot_mind_2.py
bot_mind.py

Нечетное кол-во ботов, новый турнир:
WAIT... /n/n
Победители круга 4
bot_mind_2.py
bot_mind.py

Нечетное кол-во ботов, новый турнир:
WAIT.../n/n
Победители круга 5
bot_mind_2.py

Winner: bot_mind_2.py
